In [1]:
#from master import MASTERModel
from master_strategy import MASTERModel
import pickle
import numpy as np
import time
import pandas as pd
import sys, os
import qlib
from qlib.data.dataset import TSDataSampler
from bisect import bisect_right
import time
import optuna
import pandas as pd

# Move up one directory from MASTER-master
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)


from alpha_101.alpha_generator import get_alpha101_table_from_db
from alpha_101.alpha_generator import generate_alphas
from alpha_101.alpha_generator import get_sentiment_table_from_db

In [ ]:
df_sentiment, df_sentiment_filter = get_sentiment_table_from_db()

In [ ]:

# Ensure the 'Date' column is in datetime format.
df_sentiment['Date'] = pd.to_datetime(df_sentiment['Date'])

# Get the sorted unique dates and tickers from df_sentiment.
unique_dates = sorted(df_sentiment['Date'].unique())
unique_tickers = sorted(df_sentiment['Ticker'].unique())

# Create a complete MultiIndex with all combinations of Date and Ticker.
full_index = pd.MultiIndex.from_product([unique_dates, unique_tickers], names=['Date', 'Ticker'])

# Set the DataFrame index to ['Date', 'Ticker'].
df_sentiment_indexed = df_sentiment.set_index(['Date', 'Ticker'])

# Reindex to the full index, filling missing rows with 0.
df_sentiment_filled = df_sentiment_indexed.reindex(full_index, fill_value=0).reset_index()

df_sentiment_filled['Ticker'] = df_sentiment_filled['Ticker'].str.replace(r'\$', '', regex=True)


In [ ]:
# Call generate_alphas() which returns (df, final_df)
df, final_df = generate_alphas(input_schema = 'datacollection',
                    input_table_name = 'stock_data',
                    save = True, 
                    output_schema = 'datacollection',
                    output_table_name = 'alpha101',
                    if_return = True)

Running function alpha001 for ADBE...
Running function alpha002 for ADBE...
Running function alpha003 for ADBE...
Running function alpha004 for ADBE...
Running function alpha006 for ADBE...
Running function alpha007 for ADBE...
Running function alpha008 for ADBE...
Running function alpha009 for ADBE...
Running function alpha010 for ADBE...
Running function alpha011 for ADBE...
Running function alpha012 for ADBE...
Running function alpha013 for ADBE...
Running function alpha014 for ADBE...
Running function alpha015 for ADBE...
Running function alpha016 for ADBE...
Running function alpha017 for ADBE...
Running function alpha018 for ADBE...
Running function alpha019 for ADBE...
Running function alpha020 for ADBE...
Running function alpha021 for ADBE...
Running function alpha022 for ADBE...
Running function alpha023 for ADBE...
Running function alpha024 for ADBE...
Running function alpha025 for ADBE...
Running function alpha026 for ADBE...
Running function alpha028 for ADBE...
Running func

/Users/lilingyan/QF5214_2025_G8/TeamThree/qlib_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Running function alpha008 for BIIB...
Running function alpha009 for BIIB...
Running function alpha010 for BIIB...
Running function alpha011 for BIIB...
Running function alpha012 for BIIB...
Running function alpha013 for BIIB...
Running function alpha014 for BIIB...
Running function alpha015 for BIIB...
Running function alpha016 for BIIB...
Running function alpha017 for BIIB...
Running function alpha018 for BIIB...
Running function alpha019 for BIIB...
Running function alpha020 for BIIB...
Running function alpha021 for BIIB...
Running function alpha022 for BIIB...
Running function alpha023 for BIIB...
Running function alpha024 for BIIB...
Running function alpha025 for BIIB...
Running function alpha026 for BIIB...
Running function alpha028 for BIIB...
Running function alpha029 for BIIB...
Running function alpha030 for BIIB...
Running function alpha031 for BIIB...
Running function alpha032 for BIIB...
Running function alpha033 for BIIB...
Running function alpha034 for BIIB...
Running func

In [ ]:
df_all, final_df_all, df_index = get_alpha101_table_from_db()

In [ ]:
company_list = ["ADBE", "AMD", "ABNB", "GOOGL", "GOOG", "AMZN", "AEP", "AMGN", 
"ADI", "ANSS", "AAPL", "AMAT", "APP", "ASML", "AZN", 
"TEAM", "ADSK", "ADP", "AXON", "BKR", "BIIB", "BKNG", "AVGO", 
"CDNS", "CDW", "CHTR", "CTAS", "CSCO", "CCEP", "CTSH", "CMCSA", 
"CPRT", "CSGP", "COST", "CRWD", "CSX", "DDOG", "DXCM","VRTX", "WBD", "WDAY", "XEL", "ZS","QCOM", 
"REGN", "ROP", "ROST", "FAST", "FTNT", "GILD" ,"ON", "PCAR", "PLTR", "PANW","PAYX", "PYPL", 
"PDD", "PEP","SBUX", "SNPS", "TTWO", "TMUS","TSLA", "TXN", "TTD", "VRSK"] 

'''
company_list = company_list = ["ADBE", "AMD", "ABNB", "GOOGL", "GOOG", "AMZN", "AEP", "AMGN", 
"ADI", "ANSS", "AAPL", "AMAT", "APP", "ARM", "ASML", "AZN", 
"TEAM", "ADSK", "ADP", "AXON", "BKR", "BIIB", "BKNG", "AVGO", 
"CDNS", "CDW", "CHTR", "CTAS", "CSCO", "CCEP", "CTSH", "CMCSA", 
"CEG", "CPRT", "CSGP", "COST", "CRWD", "CSX", "DDOG", "DXCM","VRTX", "WBD", "WDAY", "XEL", "ZS","QCOM", 
"REGN", "ROP", "ROST", "FAST", "FTNT", "GEHC", "GILD" ,"ON", "PCAR", "PLTR", "PANW","PAYX", "PYPL", 
"PDD", "PEP","SBUX", "SNPS", "TTWO", "TMUS","TSLA", "TXN", "TTD", "VRSK"]      '''


'\ncompany_list = company_list = ["ADBE", "AMD", "ABNB", "GOOGL", "GOOG", "AMZN", "AEP", "AMGN", \n"ADI", "ANSS", "AAPL", "AMAT", "APP", "ARM", "ASML", "AZN", \n"TEAM", "ADSK", "ADP", "AXON", "BKR", "BIIB", "BKNG", "AVGO", \n"CDNS", "CDW", "CHTR", "CTAS", "CSCO", "CCEP", "CTSH", "CMCSA", \n"CEG", "CPRT", "CSGP", "COST", "CRWD", "CSX", "DDOG", "DXCM","VRTX", "WBD", "WDAY", "XEL", "ZS","QCOM", \n"REGN", "ROP", "ROST", "FAST", "FTNT", "GEHC", "GILD" ,"ON", "PCAR", "PLTR", "PANW","PAYX", "PYPL", \n"PDD", "PEP","SBUX", "SNPS", "TTWO", "TMUS","TSLA", "TXN", "TTD", "VRSK"]      '

In [ ]:
filtered_df_all = df_all[df_all['Ticker'].isin(company_list)]
filtered_final_df_all = final_df_all[final_df_all['Ticker'].isin(company_list)]
filtered_df_sentiment = df_sentiment_filled[df_sentiment_filled['Ticker'].isin(company_list)]

In [ ]:
filtered_df_all.to_csv("data/Input/stock.csv", index=False)
filtered_final_df_all.to_csv("data/Input/alpha.csv", index=False)
df_index.to_csv("data/Input/index.csv", index=False)
filtered_df_sentiment.to_csv("data/Input/sentiment.csv", index=False)

In [38]:
df_index

,Date,Open,High,Low,Close,Adj_Close,Volume,Ticker,IndexName
0,2022-01-03,4778.140137,4796.640137,4758.169922,4796.560059,4796.560059,3.831020e+09,^GSPC,S&P 500
1,2022-01-04,4804.509766,4818.620117,4774.270020,4793.540039,4793.540039,4.683170e+09,^GSPC,S&P 500
2,2022-01-05,4787.990234,4797.700195,4699.439941,4700.580078,4700.580078,4.887960e+09,^GSPC,S&P 500
3,2022-01-06,4693.390137,4725.009766,4671.259766,4696.049805,4696.049805,4.295280e+09,^GSPC,S&P 500
4,2022-01-07,4697.660156,4707.950195,4662.740234,4677.029785,4677.029785,4.181510e+09,^GSPC,S&P 500
...,...,...,...,...,...,...,...,...,...
4049,2025-03-28,5679.200195,5685.890137,5572.419922,5580.939941,5580.939941,4.513330e+09,^GSPC,S&P 500
4050,2025-03-28,19718.460938,19752.449219,19241.539062,19281.400391,19281.400391,7.015410e+09,^NDX,NASDAQ 100
4051,2025-03-28,3108.610107,3108.790039,3046.620117,3051.560059,3051.560059,0.000000e+00,^RUI,Russell 1000
4052,2025-03-28,3233.379883,3233.399902,3168.500000,3173.889893,3173.889893,0.000000e+00,^RUA,Russell 3000


In [37]:
df_all = pd.read_csv("data/Input/stock.csv")
final_df_all = pd.read_csv("data/Input/alpha.csv")
df_index = pd.read_csv("data/Input/index.csv")
df_sentiment_filled = pd.read_csv("data/Input/sentiment.csv")

In [39]:
# Ensure the Date column is in datetime format.
df_all['Date'] = pd.to_datetime(df_all['Date'])
final_df_all['Date'] = pd.to_datetime(final_df_all['Date'])
df_index['Date'] = pd.to_datetime(df_index['Date'])
df_sentiment_filled['Date'] = pd.to_datetime(df_sentiment_filled['Date'])

# Filter the DataFrame: select rows where Date is on or before 2025-02-28.
filtered_df_all = df_all[df_all['Date'] <= pd.Timestamp('2025-02-28')]
filtered_final_df_all = final_df_all[final_df_all['Date'] <= pd.Timestamp('2025-02-28')]
filtered_df_index = df_index[df_index['Date'] <= pd.Timestamp('2025-02-28')]
filtered_df_sentiment = df_sentiment_filled[df_sentiment_filled['Date'] <= pd.Timestamp('2025-02-28')]

In [40]:
# 1) Drop the "IndexTicker" and "Volume" column if it exists
filtered_df_index = filtered_df_index.drop(columns="IndexTicker", errors="ignore")
filtered_df_index = filtered_df_index.drop(columns="Volume", errors="ignore")

# 2) Pivot the DataFrame
df_index_pivot = filtered_df_index.pivot(
    index="Date",                # Each Date becomes a row
    columns="IndexName",         # Each unique IndexName becomes columns
    values=["Open", "High", "Low", "Close", "Adj_Close"]
)

# 3) Flatten and rename columns to single-level format "<IndexName>_<Column>"
df_index_pivot.columns = [
    f"{col[1]}_{col[0]}"  # col is a tuple like ("Open", "S&P500")
    for col in df_index_pivot.columns
]

# 4) Move the 'Date' index back into a column
df_index_pivot.reset_index(inplace=True)


df_index_pivot

,Date,NASDAQ 100_Open,Russell 1000_Open,Russell 3000_Open,S&P 500_Open,Wilshire 5000_Open,NASDAQ 100_High,Russell 1000_High,Russell 3000_High,S&P 500_High,...,NASDAQ 100_Close,Russell 1000_Close,Russell 3000_Close,S&P 500_Close,Wilshire 5000_Close,NASDAQ 100_Adj_Close,Russell 1000_Adj_Close,Russell 3000_Adj_Close,S&P 500_Adj_Close,Wilshire 5000_Adj_Close
0,2022-01-03,16395.509766,2645.679932,2788.020020,4778.140137,48461.160156,16504.140625,2660.949951,2804.939941,4796.640137,...,16501.769531,2660.780029,2804.929932,4796.560059,48789.359375,16501.769531,2660.780029,2804.929932,4796.560059,48789.359375
1,2022-01-04,16513.869141,2660.989990,2788.020020,4804.509766,48789.359375,16513.869141,2672.000000,2817.110107,4818.620117,...,16279.730469,2656.489990,2800.409912,4793.540039,48660.750000,16279.730469,2656.489990,2800.409912,4793.540039,48660.750000
2,2022-01-05,16190.549805,2655.800049,2799.729980,4787.990234,48660.750000,16249.230469,2657.149902,2801.260010,4797.700195,...,15771.780273,2600.530029,2739.340088,4700.580078,47534.621094,15771.780273,2600.530029,2739.340088,4700.580078,47534.621094
3,2022-01-06,15675.219727,2600.379883,2739.229980,4693.390137,47534.621094,15900.599609,2616.030029,2756.689941,4725.009766,...,15765.360352,2599.429932,2739.199951,4696.049805,47542.789062,15765.360352,2599.429932,2739.199951,4696.049805,47542.789062
4,2022-01-07,15774.129883,2599.300049,2739.060059,4697.660156,47542.789062,15835.000000,2607.590088,2748.270020,4707.950195,...,15592.190430,2587.729980,2725.600098,4677.029785,47321.140625,15592.190430,2587.729980,2725.600098,4677.029785,47321.140625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,2025-02-24,21669.359375,3298.969971,3432.129883,6026.689941,60137.691406,21727.580078,3306.330078,3439.399902,6043.649902,...,21352.080078,3274.929932,3406.699951,5983.250000,59821.519531,21352.080078,3274.929932,3406.699951,5983.250000,59821.519531
788,2025-02-25,21318.769531,3274.979980,3406.899902,5982.729980,59821.519531,21326.810547,3279.629883,3411.750000,5992.649902,...,21087.250000,3258.209961,3389.510010,5955.250000,59519.980469,21087.250000,3258.209961,3389.510010,5955.250000,59519.980469
789,2025-02-26,21170.509766,3262.580078,3394.050049,5970.870117,59519.980469,21350.519531,3289.090088,3422.129883,6009.819824,...,21132.919922,3259.419922,3391.000000,5956.060059,59547.570312,21132.919922,3259.419922,3391.000000,5956.060059,59547.570312
790,2025-02-27,21281.359375,3266.239990,3397.850098,5981.879883,59547.570312,21323.910156,3279.389893,3410.889893,5993.689941,...,20550.949219,3208.979980,3338.459961,5861.569824,58616.808594,20550.949219,3208.979980,3338.459961,5861.569824,58616.808594


In [42]:
# Merge on "Date" and "Ticker" (adjust join type if needed)
combined_df = pd.merge(filtered_df_all, filtered_final_df_all, on=["Date", "Ticker"], how="inner")

In [43]:
combined_df

,Date,Open,High,Low,Close,Volume,Market_Cap,Ticker,IndClass_Sector,IndClass_Industry,...,80,81,82,83,84,85,86,87,88,89
0,2022-01-03,566.650024,575.000000,553.270020,564.369995,2862700.0,1.918454e+11,ADBE,Technology,Software - Application,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.860837,0.0,-0.104921
1,2022-01-04,563.900024,564.299988,541.309998,554.000000,4344800.0,1.918454e+11,ADBE,Technology,Software - Application,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.860837,0.0,-0.430605
2,2022-01-05,535.000000,538.750000,514.000000,514.429993,7394800.0,1.918454e+11,ADBE,Technology,Software - Application,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.860837,0.0,-0.831078
3,2022-01-06,510.000000,526.309998,508.200012,514.119995,3913600.0,1.918454e+11,ADBE,Technology,Software - Application,...,0.006158,NaN,NaN,NaN,NaN,NaN,NaN,0.860837,0.0,0.227486
4,2022-01-07,513.659973,520.419983,504.000000,510.700012,3605400.0,1.918454e+11,ADBE,Technology,Software - Application,...,0.431034,NaN,NaN,NaN,NaN,NaN,NaN,0.860837,0.0,-0.180255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52267,2025-02-21,205.929993,206.330002,203.619995,204.429993,2302600.0,7.787188e+10,CTAS,Industrials,Specialty Business Services,...,0.756158,3.0,7.0,11.028862,-0.657465,-8.0,6.278325,-0.380542,0.0,-0.553300
52268,2025-02-24,204.619995,206.470001,203.199997,203.350006,1613700.0,7.787188e+10,CTAS,Industrials,Specialty Business Services,...,0.928571,6.0,7.0,13.791932,-0.607277,-5.0,6.328818,-0.288177,0.0,-0.388257
52269,2025-02-25,202.240005,207.490005,202.240005,205.820007,2206000.0,7.787188e+10,CTAS,Industrials,Specialty Business Services,...,0.217980,2.0,7.0,14.371681,-0.344806,-4.0,6.422414,-0.113300,0.0,0.681775
52270,2025-02-26,203.389999,207.520004,202.570007,204.309998,1519200.0,7.787188e+10,CTAS,Industrials,Specialty Business Services,...,0.087438,0.0,7.0,15.393365,-0.132191,-6.0,6.483990,-0.003079,0.0,0.185821


In [44]:
filtered_df_sentiment = filtered_df_sentiment.drop(columns=["Intent Sentiment"])

for col in filtered_df_sentiment.columns:
    if col not in ['Date', 'Ticker']:
        filtered_df_sentiment[col] = pd.to_numeric(filtered_df_sentiment[col], errors='coerce')

In [45]:
# Get the sorted unique trading dates from df_merged.
trading_dates = sorted(combined_df['Date'].unique())
    
combined_sent_list = []

In [46]:
# Process each ticker separately.
tickers = combined_df['Ticker'].unique()
for ticker in tickers:
    # Subset sentiment data for this ticker.
    df_sent_ticker = filtered_df_sentiment[filtered_df_sentiment['Ticker'] == ticker].copy()
    # Loop over each trading date.
    for i, current_date in enumerate(trading_dates):
        if i < len(trading_dates) - 1:
            next_date = trading_dates[i+1]
        else:
            next_date = None
        
        # Define the window: from current_date (inclusive) to next_date (exclusive).
        if next_date is not None:
            window = df_sent_ticker[(df_sent_ticker['Date'] >= current_date) & 
                                        (df_sent_ticker['Date'] < next_date)]
        else:
            window = df_sent_ticker[df_sent_ticker['Date'] >= current_date]
        
        # If there's sentiment data in the window, compute the average.
        if not window.empty:
            # Assume sentiment columns are all except 'Date' and 'Ticker'.
            sentiment_cols = [col for col in window.columns if col not in ['Date', 'Ticker']]
            avg_values = window[sentiment_cols].mean()
            
            # Create a new row with the current trading date, ticker, and averaged sentiment.
            row = {'Date': current_date, 'Ticker': ticker}
            row.update(avg_values.to_dict())
            combined_sent_list.append(row)

In [47]:
tickers

array(['ADBE', 'AMD', 'ABNB', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN',
       'ADI', 'ANSS', 'AAPL', 'AMAT', 'APP', 'ASML', 'AZN', 'TEAM',
       'ADSK', 'ADP', 'AXON', 'BKR', 'BIIB', 'BKNG', 'AVGO', 'CDNS',
       'CDW', 'CHTR', 'CSCO', 'CCEP', 'CTSH', 'CMCSA', 'CPRT', 'CSGP',
       'COST', 'CRWD', 'CSX', 'DDOG', 'DXCM', 'FAST', 'FTNT', 'GILD',
       'ON', 'PCAR', 'PLTR', 'PANW', 'PAYX', 'PYPL', 'PDD', 'PEP', 'QCOM',
       'REGN', 'ROP', 'ROST', 'SBUX', 'SNPS', 'TTWO', 'TMUS', 'TSLA',
       'TXN', 'TTD', 'VRSK', 'VRTX', 'WBD', 'WDAY', 'XEL', 'ZS', 'CTAS'],
      dtype=object)

In [48]:
# Create a DataFrame from the combined sentiment data.
df_combined_sent = pd.DataFrame(combined_sent_list)


# Drop any rows whose dates are not in df_merged (should be none if we use trading_dates,
    # but we do this for safety).
df_combined_sent = df_combined_sent[df_combined_sent['Date'].isin(combined_df['Date'])]
    

# Merge df_merged with the combined sentiment data.
# Use a left merge so that all trading dates and tickers in df_merged are preserved.
df_final = pd.merge(combined_df, df_combined_sent, on=['Date', 'Ticker'], how='left')



In [49]:
df_final

,Date,Open,High,Low,Close,Volume,Market_Cap,Ticker,IndClass_Sector,IndClass_Industry,...,89,Positive,Negative,Neutral,Surprise,Joy,Anger,Fear,Sadness,Disgust
0,2022-01-03,566.650024,575.000000,553.270020,564.369995,2862700.0,1.918454e+11,ADBE,Technology,Software - Application,...,-0.104921,0.220803,0.104503,0.674694,0.286160,0.354057,0.082727,0.189870,0.043234,0.043952
1,2022-01-04,563.900024,564.299988,541.309998,554.000000,4344800.0,1.918454e+11,ADBE,Technology,Software - Application,...,-0.430605,0.087670,0.325933,0.586397,0.194531,0.166576,0.158554,0.274148,0.179535,0.026657
2,2022-01-05,535.000000,538.750000,514.000000,514.429993,7394800.0,1.918454e+11,ADBE,Technology,Software - Application,...,-0.831078,0.141726,0.203461,0.654812,0.233590,0.322107,0.148563,0.108585,0.176740,0.010415
3,2022-01-06,510.000000,526.309998,508.200012,514.119995,3913600.0,1.918454e+11,ADBE,Technology,Software - Application,...,0.227486,0.224925,0.133087,0.641988,0.181662,0.194559,0.194219,0.176187,0.203045,0.050328
4,2022-01-07,513.659973,520.419983,504.000000,510.700012,3605400.0,1.918454e+11,ADBE,Technology,Software - Application,...,-0.180255,0.218923,0.198734,0.582343,0.160200,0.463430,0.116098,0.076089,0.170252,0.013931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52267,2025-02-21,205.929993,206.330002,203.619995,204.429993,2302600.0,7.787188e+10,CTAS,Industrials,Specialty Business Services,...,-0.553300,0.197417,0.042255,0.426995,0.130383,0.458995,0.032484,0.011738,0.029106,0.003960
52268,2025-02-24,204.619995,206.470001,203.199997,203.350006,1613700.0,7.787188e+10,CTAS,Industrials,Specialty Business Services,...,-0.388257,0.311825,0.166780,0.521395,0.151385,0.483016,0.170509,0.049247,0.137808,0.008035
52269,2025-02-25,202.240005,207.490005,202.240005,205.820007,2206000.0,7.787188e+10,CTAS,Industrials,Specialty Business Services,...,0.681775,0.014371,0.960115,0.025514,0.063958,0.356006,0.039598,0.012111,0.522017,0.006310
52270,2025-02-26,203.389999,207.520004,202.570007,204.309998,1519200.0,7.787188e+10,CTAS,Industrials,Specialty Business Services,...,0.185821,0.171986,0.038720,0.789294,0.188029,0.475572,0.103876,0.027684,0.190603,0.014236


In [50]:

# 5) (Optional) Merge with your main DataFrame on 'Date'
df_merged = pd.merge(
    df_final,
    df_index_pivot,
    on='Date',    # or how='left'/'right'/'outer' if needed
    how='left'    
)

df_merged.fillna(0, inplace=True)

# Example of dropping non-numeric columns:
filtered_df_merged = df_merged.drop(columns=["IndClass_Sector", "IndClass_Industry"])

In [52]:

df_sorted = filtered_df_merged.sort_values(by='Date').reset_index(drop=True)

df_sorted_shift = df_sorted.sort_values(["Ticker", "Date"])

# 2) Shift the Return column so that row T holds day T+1's label
df_sorted_shift["Return"] = df_sorted_shift.groupby("Ticker")["Return"].shift(-1)

# 3) Drop rows where Return became NaN after shifting
df_sorted_shift.dropna(subset=["Return"], inplace=True)

In [53]:
# Ensure the Date column is datetime type.
df_sorted_shift['Date'] = pd.to_datetime(df_sorted_shift['Date'])

# Get the sorted unique dates.
unique_dates = np.sort(df_sorted_shift['Date'].unique())

# Define split indices based on unique dates.
n_dates = len(unique_dates)
train_date_end = unique_dates[int(n_dates * 0.6)]
valid_date_end = unique_dates[int(n_dates * 0.8)]

# Now split the DataFrame based on the date thresholds.
df_train = df_sorted_shift[df_sorted_shift['Date'] <= train_date_end].copy()
df_valid = df_sorted_shift[(df_sorted_shift['Date'] > train_date_end) & (df_sorted_shift['Date'] <= valid_date_end)].copy()
df_test  = df_sorted_shift[(df_sorted_shift['Date'] > valid_date_end)].copy()


In [54]:
df_train

,Date,Open,High,Low,Close,Volume,Market_Cap,Ticker,Typical_Price,Return,...,NASDAQ 100_Close,Russell 1000_Close,Russell 3000_Close,S&P 500_Close,Wilshire 5000_Close,NASDAQ 100_Adj_Close,Russell 1000_Adj_Close,Russell 3000_Adj_Close,S&P 500_Adj_Close,Wilshire 5000_Adj_Close
39,2022-01-03,177.830002,182.880005,177.710007,182.009995,104487900.0,3.575710e+12,AAPL,180.866669,-0.012692,...,16501.769531,2660.780029,2804.929932,4796.560059,48789.359375,16501.769531,2660.780029,2804.929932,4796.560059,48789.359375
79,2022-01-04,182.630005,182.940002,179.119995,179.699997,99310400.0,3.575710e+12,AAPL,180.586665,-0.026600,...,16279.730469,2656.489990,2800.409912,4793.540039,48660.750000,16279.730469,2656.489990,2800.409912,4793.540039,48660.750000
196,2022-01-05,179.610001,180.169998,174.639999,174.919998,94537600.0,3.575710e+12,AAPL,176.576665,-0.016693,...,15771.780273,2600.530029,2739.340088,4700.580078,47534.621094,15771.780273,2600.530029,2739.340088,4700.580078,47534.621094
206,2022-01-06,172.699997,175.300003,171.639999,172.000000,96904000.0,3.575710e+12,AAPL,172.980001,0.000988,...,15765.360352,2599.429932,2739.199951,4696.049805,47542.789062,15765.360352,2599.429932,2739.199951,4696.049805,47542.789062
290,2022-01-07,172.889999,174.139999,171.029999,172.169998,86709100.0,3.575710e+12,AAPL,172.446665,0.000116,...,15592.190430,2587.729980,2725.600098,4677.029785,47321.140625,15592.190430,2587.729980,2725.600098,4677.029785,47321.140625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31072,2023-11-15,188.000000,189.539001,183.464996,184.559998,1954000.0,2.896775e+10,ZS,185.854665,0.008778,...,15817.179688,2463.370117,2574.290039,4502.879883,45224.910156,15817.179688,2463.370117,2574.290039,4502.879883,45224.910156
31091,2023-11-16,183.720001,189.300003,183.139999,186.179993,1701300.0,2.896775e+10,ZS,186.206665,0.006929,...,15833.169922,2464.060059,2572.969971,4508.240234,45200.058594,15833.169922,2464.060059,2572.969971,4508.240234,45200.058594
31184,2023-11-17,186.029999,189.710007,185.520004,187.470001,1293800.0,2.896775e+10,ZS,187.566671,0.023951,...,15837.990234,2468.760010,2579.379883,4514.020020,45312.960938,15837.990234,2468.760010,2579.379883,4514.020020,45312.960938
31233,2023-11-20,188.869995,193.490005,188.869995,191.960007,1569100.0,2.896775e+10,ZS,191.440002,-0.003126,...,16027.059570,2487.000000,2598.149902,4547.379883,45642.968750,16027.059570,2487.000000,2598.149902,4547.379883,45642.968750


In [55]:
def convert_data_qlibformat(df):

    # Create a copy to avoid SettingWithCopyWarning
    df = df.copy()
    
    # remove rows with missing "Return" values
    df.dropna(subset=['Return'], inplace=True)
    
    # 1) Ensure the "Date" column is a datetime type
    df["Date"] = pd.to_datetime(df["Date"])
    
    # 2) Separate label and features BEFORE setting the index
    #    Retain "Date" and "Ticker" for indexing, but remove them from features.
    df_label = df[["Return"]].copy()
    # Exclude "Date" and "Ticker" along with "Return" from features
    df_feature = df.drop(columns=["Return", "Date", "Ticker"], errors="ignore")
    
    # 3) Normalize feature columns (ensure they are numeric)
    df_feature = df_feature.apply(pd.to_numeric, errors='coerce')
    feature_mean = df_feature.mean()
    feature_std = df_feature.std()
    eps = 1e-8  # to avoid division by zero
    normalized_feature = (df_feature - feature_mean) / (feature_std + eps)
    df_feature = normalized_feature
    
    # 4) Now set the index of the original DataFrame using "Date" and "Ticker"
    #    These columns will be used as the index ("datetime" and "instrument")
    df_index = df.set_index(["Date", "Ticker"])
    
    # 5) Assign the index to both label and feature DataFrames
    df_feature.index = df_index.index
    df_label.index = df_index.index

    # 6) Construct MultiIndex for the columns:
    #    For features, use top-level "feature"; for the label, use "label"
    df_feature.columns = pd.MultiIndex.from_product([["feature"], df_feature.columns])
    df_label.columns = pd.MultiIndex.from_product([["label"], df_label.columns])
    
    # 7) Concatenate features and label columns, and fill missing values with 0
    df_qlib = pd.concat([df_feature, df_label], axis=1)
    df_qlib = df_qlib.fillna(0)
    
    # 8) Name the index levels and sort the DataFrame
    df_qlib.index.names = ["datetime", "instrument"]
    df_qlib = df_qlib.sort_index(level=["datetime", "instrument"])

    
    # 9) Determine start and end dates from the datetime index level
    start = df_qlib.index.get_level_values("datetime").min()
    end = df_qlib.index.get_level_values("datetime").max()
    
    # 10) Build TSDataSampler (using fillna_type='ffill+bfill' for reindexing)
    sampler = TSDataSampler(df_qlib, start, end, step_len=8, fillna_type='ffill+bfill')
    sampler.data_arr = np.nan_to_num(sampler.data_arr, nan=0.0)
    
    return sampler

# Use the function for training, validation, and test sets:
dl_train = convert_data_qlibformat(df_train)
dl_valid = convert_data_qlibformat(df_valid)
dl_test = convert_data_qlibformat(df_test)


In [56]:
# Save the merged DataFrame tinpuo a pickle file
with open("data/input/training_sentiment_input.pkl", "wb") as f:
    pickle.dump(dl_train, f)
with open("data/input/valid_sentiment_input.pkl", "wb") as f:
    pickle.dump(dl_valid, f)
with open("data/input/testing_sentiment_input.pkl", "wb") as f:
    pickle.dump(dl_test, f)


In [57]:
sorted(df_test['Date'])

[Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-12 00:00:00'),
 Timestamp('2024-07-

In [ ]:
# After loading:
with open('data/Input/training_sentiment_input.pkl', 'rb') as f:
    ts_sampler = pickle.load(f)

print(type(ts_sampler))
print(dir(ts_sampler))
print(ts_sampler.__dict__)

# Extract the data array from the sampler
data_arr = ts_sampler.data_arr

# Convert the data array to a DataFrame
data_df = pd.DataFrame(data_arr)

# Display the DataFrame
print(data_df.head())

# Check for any NaN values in the data array
if np.isnan(data_arr).any():
    print("NaN found in ts_sampler.data_arr")
else:
    print("No NaN found in ts_sampler.data_arr")

<class 'qlib.data.dataset.TSDataSampler'>
['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_get_indices', '_get_row_col', 'build_index', 'config', 'data_arr', 'data_index', 'empty', 'end', 'fillna_type', 'flt_idx_map', 'get_index', 'idx_arr', 'idx_df', 'idx_map', 'idx_map2arr', 'nan_idx', 'slice_idx_map_and_data_index', 'start', 'step_len']
{'start': Timestamp('2022-01-03 00:00:00'), 'end': Timestamp('2023-11-21 00:00:00'), 'step_len': 8, 'fillna_type': 'ffill+bfill', 'data_arr': array([[-0.05410935, -0.04598868, -0.04642936, ..., -0.27496841,
        -0.20114885, -0.0126916 ],
       [-0.03865579, -0.04579801, -0.041

In [ ]:
for i, batch in enumerate(dl_train):
    if i == 3:
        print(f"Batch {i}:")
        print("  Type:", type(batch))            # Should be <class 'numpy.ndarray'>
        print("  Shape:", batch.shape)           # e.g. (batch_size, step_len, feature_dim)
        print("  Sample data:", batch[:2])         # First 2 samples
    if i >= 3:
        break



Batch 3:
  Type: <class 'numpy.ndarray'>
  Shape: (8, 131)
  Sample data: [[-5.41093521e-02 -4.59886797e-02 -4.64293626e-02 -4.08015421e-02
   4.07723437e+00  5.51328787e+00 -4.44059912e-02 -6.23283231e-01
  -1.62980000e+00 -6.79906337e-02 -1.53982414e-01  1.63402835e+00
  -1.46431198e-01  1.06744131e-02  1.66983819e+00 -2.33320478e-03
  -3.86856395e-03 -1.16487749e+00 -1.77047739e-02  1.67258454e+00
  -1.31676441e-01  2.43861389e+00  1.66164425e+00  7.72775405e-01
   1.70471217e+00  9.07592925e-02  8.17382150e-01  9.73601895e-01
   7.11885094e-03  1.59903654e-01  2.25838845e-01 -1.63649857e+00
   6.22451227e-01 -1.09438816e+00 -2.12026921e+00 -1.51845384e+00
  -9.41428258e-01 -1.01477791e+00 -1.47423376e+00 -1.73656002e+00
  -6.68186587e-01 -1.06916418e+00 -9.86110691e-01  1.09829848e+00
   7.39521924e-01  1.74818955e-02 -1.01271434e-02 -1.84531815e-01
  -1.00983120e+00  8.85706159e-02 -2.94790332e-02 -6.44288872e-02
  -6.34789181e-01  2.31501122e-03 -1.61108073e-02  3.27204915e+00
  

Hyperparameters tuning

In [66]:
with open(f'data/input/training_sentiment_input.pkl', 'rb') as f:
    dl_train = pickle.load(f)
with open(f'data/input/valid_sentiment_input.pkl', 'rb') as f:
    dl_valid = pickle.load(f)
with open(f'data/input/testing_sentiment_input.pkl', 'rb') as f:
    dl_test = pickle.load(f)

In [67]:
import time
import itertools
import pandas as pd

# Define your hyperparameter lists
d_model_list   = [128, 256, 512]
t_nhead_list   = [4]
s_nhead_list   = [2]
dropout_list   = [0.7]
beta_list      = [5]
lr_list        = [1e-3, 1e-4, 1e-5]


# Fixed parameters
d_feat = 106
gate_input_start_index = 106
gate_input_end_index = 121 + 9
n_epoch = 100
GPU = 0
train_stop_loss_thred = 0.0007

# Prepare a list to store results for each hyperparameter combination
results = []

# Loop over all combinations using itertools.product
for d_model, t_nhead, s_nhead, dropout, beta, lr in itertools.product(
    d_model_list, t_nhead_list, s_nhead_list, dropout_list, beta_list, lr_list
):
    print(f"Running combination: d_model={d_model}, t_nhead={t_nhead}, s_nhead={s_nhead}, dropout={dropout}, beta={beta}, lr={lr}")
    
    # To average results over seeds; here we use one seed but you can add more if needed
    ic_list = []
    icir_list = []
    ric_list = []
    ricir_list = []
    
    for seed in [0]:
        # Initialize your model with the current hyperparameter combination
        model = MASTERModel(
            d_feat=d_feat,
            d_model=d_model,
            t_nhead=t_nhead,
            s_nhead=s_nhead,
            T_dropout_rate=dropout,
            S_dropout_rate=dropout,
            beta=beta,
            gate_input_end_index=gate_input_end_index,
            gate_input_start_index=gate_input_start_index,
            n_epochs=n_epoch,
            lr=lr,
            GPU=GPU,
            seed=seed,
            train_stop_loss_thred=train_stop_loss_thred,
            save_path='model',
            save_prefix='model_training'
        )
    
        start = time.time()
        # Train the model using your training code
        model.fit(dl_train, df_all, dl_valid)
        print("Model Trained.")
    
        # Prediction / Testing phase
        print("Start Prediction")
        predictions, metrics, real_returns, real_prices, market_cap = model.predict(dl_test, df_all)
        run_time = time.time() - start
        print(f"Seed: {seed} time cost: {run_time:.2f} sec")
        print("Metrics:", metrics)
    
        # Append each metric after converting to float
        ic_list.append(float(metrics['IC']))
        icir_list.append(float(metrics['ICIR']))
        ric_list.append(float(metrics['RIC']))
        ricir_list.append(float(metrics['RICIR']))
    
    # Compute mean values for each indicator if available
    mean_ic   = np.mean(ic_list)    if ic_list   else None
    mean_icir = np.mean(icir_list)  if icir_list else None
    mean_ric  = np.mean(ric_list)    if ric_list  else None
    mean_ricir= np.mean(ricir_list)  if ricir_list else None
    
    # Save the hyperparameter combination and its performance
    results.append({
        "d_model": d_model,
        "t_nhead": t_nhead,
        "s_nhead": s_nhead,
        "dropout": dropout,
        "beta": beta,
        "lr": lr,
        "mean_ic": mean_ic,
        'mean_ICIR': mean_icir,
        'mean_RIC': mean_ric,
        'mean_RICIR': mean_ricir
    })
    
# Convert the results to a DataFrame and save to CSV
df_results = pd.DataFrame(results)
df_results.to_csv("data/Output/hyperparameter_sentiment_results.csv", index=False)
print("Grid search results saved to data/Output/hyperparameter_sentiment_results.csv")


Running combination: d_model=128, t_nhead=4, s_nhead=2, dropout=0.7, beta=5, lr=0.001
Epoch: 0
Epoch 0, train_loss 0.023776, valid ic 0.0421, icir 0.389, rankic 0.0314, rankicir 0.289.
Epoch: 1
Epoch 1, train_loss 0.001215, valid ic 0.0628, icir 0.594, rankic 0.0520, rankicir 0.478.
Epoch: 2
Epoch 2, train_loss 0.001032, valid ic 0.0502, icir 0.414, rankic 0.0357, rankicir 0.307.
Epoch: 3
Epoch 3, train_loss 0.000885, valid ic 0.0703, icir 0.613, rankic 0.0568, rankicir 0.518.
Epoch: 4
Epoch 4, train_loss 0.000847, valid ic 0.0876, icir 0.782, rankic 0.0781, rankicir 0.738.
Epoch: 5
Epoch 5, train_loss 0.000898, valid ic 0.0607, icir 0.486, rankic 0.0468, rankicir 0.393.
Epoch: 6
Epoch 6, train_loss 0.000827, valid ic 0.0700, icir 0.566, rankic 0.0562, rankicir 0.473.
Epoch: 7
Epoch 7, train_loss 0.000794, valid ic 0.0540, icir 0.470, rankic 0.0383, rankicir 0.322.
Epoch: 8
Epoch 8, train_loss 0.000757, valid ic 0.0859, icir 0.769, rankic 0.0770, rankicir 0.715.
Epoch: 9
Epoch 9, train

Final hyperparametrs

In [99]:

    
d_feat = 106
d_model = 512
t_nhead = 4
s_nhead = 2
dropout = 0.7
gate_input_start_index = 106
gate_input_end_index = 121 + 9

beta = 5

n_epoch = 100
lr = 1e-4
GPU = 0
train_stop_loss_thred = 0.00075


ic = []
icir = []
ric = []
ricir = []

In [100]:
def get_next_trading_day(current_date, trading_dates):
    """
    Given a current_date and a sorted list of trading_dates,
    returns the next trading day after current_date.
    If no next trading day exists, returns None.
    """
    pos = bisect_right(trading_dates, current_date)
    if pos < len(trading_dates):
        return trading_dates[pos]
    else:
        return None

In [101]:
##Training
######################################################################################
for seed in [0]: # , 1, 2, 3, 4
    model = MASTERModel(
        d_feat = d_feat, d_model = d_model, t_nhead = t_nhead, s_nhead = s_nhead, T_dropout_rate=dropout, S_dropout_rate=dropout,
        beta=beta, gate_input_end_index=gate_input_end_index, gate_input_start_index=gate_input_start_index,
        n_epochs=n_epoch, lr = lr, GPU = GPU, seed = seed, train_stop_loss_thred = train_stop_loss_thred,
        save_path='model', save_prefix=f'model_training_with_sentiment'
    )

    start = time.time()
    # Train
    model.fit(dl_train, df_all, dl_valid)

    print("Model Trained.")

    # Test
    # predictions, metrics = model.predict(dl_test)
    ###### 0
    print("Start Prediction")
    predictions, metrics, real_returns, real_prices, market_cap = model.predict(dl_test, df_all)
    ######       
    running_time = time.time()-start
    
    print('Seed: {:d} time cost : {:.2f} sec'.format(seed, running_time))
    print(metrics)

    ic.append(metrics['IC'])
    icir.append(metrics['ICIR'])
    ric.append(metrics['RIC'])
    ricir.append(metrics['RICIR'])

    
######################################################################################



Epoch: 0
Epoch 0, train_loss 0.060142, valid ic 0.0545, icir 0.454, rankic 0.0463, rankicir 0.350.
Epoch: 1
Epoch 1, train_loss 0.004883, valid ic 0.0845, icir 0.779, rankic 0.0737, rankicir 0.631.
Epoch: 2
Epoch 2, train_loss 0.007497, valid ic 0.0811, icir 0.764, rankic 0.0697, rankicir 0.673.
Epoch: 3
Epoch 3, train_loss 0.001963, valid ic 0.0639, icir 0.568, rankic 0.0551, rankicir 0.477.
Epoch: 4
Epoch 4, train_loss 0.001481, valid ic 0.1041, icir 0.973, rankic 0.0985, rankicir 0.936.
Epoch: 5
Epoch 5, train_loss 0.001335, valid ic 0.0296, icir 0.254, rankic 0.0229, rankicir 0.191.
Epoch: 6
Epoch 6, train_loss 0.001669, valid ic 0.0537, icir 0.463, rankic 0.0493, rankicir 0.398.
Epoch: 7
Epoch 7, train_loss 0.001061, valid ic 0.0573, icir 0.504, rankic 0.0518, rankicir 0.425.
Epoch: 8
Epoch 8, train_loss 0.001052, valid ic 0.0719, icir 0.596, rankic 0.0618, rankicir 0.501.
Epoch: 9
Epoch 9, train_loss 0.001233, valid ic 0.0847, icir 0.742, rankic 0.0788, rankicir 0.658.
Epoch: 10


In [106]:
# Load and Test
######################################################################################
for seed in [0]:
    param_path = f'model/model_training_with_sentiment_{seed}.pkl'

    print(f'Model Loaded from {param_path}')
    model = MASTERModel(
            d_feat = d_feat, d_model = d_model, t_nhead = t_nhead, s_nhead = s_nhead, T_dropout_rate=dropout, S_dropout_rate=dropout,
            beta=beta, gate_input_end_index=gate_input_end_index, gate_input_start_index=gate_input_start_index,
            n_epochs=n_epoch, lr = lr, GPU = GPU, seed = seed, train_stop_loss_thred = train_stop_loss_thred,
            save_path='model', save_prefix=f'model_prediction_with_sentiment'
        )
    model.load_param(param_path)
    ###### 1
    predictions, metrics, real_returns, real_prices, market_cap = model.predict(dl_test, df_all)
    ###### 
    # predictions, metrics = model.predict(dl_test)
    print(metrics)

    ic.append(metrics['IC'])
    icir.append(metrics['ICIR'])
    ric.append(metrics['RIC'])
    ricir.append(metrics['RICIR'])

    ####### 2
    # From Predictions to DataFrame
    df_predictions = predictions.reset_index()  # 让 MultiIndex 变成普通列
    df_predictions.columns = ["Date", "Ticker", "Predicted_Return"]  # Rename Column

    # Get a sorted list of unique trading dates.
    trading_dates = sorted(df_predictions['Date'].unique())


    # Now assume df_predictions has a column "Date" corresponding to the last day of the input window.
    # Create a new column "Predicted_Return_Date" which is the next trading day.
    df_predictions['Date'] = df_predictions['Date'].apply(
        lambda d: get_next_trading_day(d, trading_dates)
    )

    # Make sure real_returns, real_prices, market_cap are Series indexed by (datetime, instrument)
    df_real_returns = real_returns.rename("Actual_Return").reset_index()
    df_real_prices = real_prices.rename("Price").reset_index()
    df_market_cap = market_cap.rename("Market_Cap").reset_index()
    #df_predictions = df_predictions.reset_index()

    df_predictions = df_predictions.dropna(subset=["Date"])

    # 2) Rename columns in df_predictions so they match the merge keys.
    df_predictions.rename(columns={"Date": "datetime", "Ticker": "instrument"}, inplace=True)

    # 3) If df_predictions already has columns like "Actual_Return", "Price", "Market_Cap"
    #    (i.e. placeholders from earlier steps), remove them to avoid conflicts during merge.
    df_predictions.drop(columns=["Actual_Return", "Price", "Market_Cap"], errors="ignore", inplace=True)

    # Now df_predictions has columns: ["datetime", "instrument", "Predicted_Return", ...]

    # 4) Merge on ["datetime", "instrument"] with how="inner" so we only keep rows that exist in both.
    df_merged_predictions = (
        df_predictions
        .merge(df_real_returns, on=["datetime", "instrument"], how="inner")
        .merge(df_real_prices, on=["datetime", "instrument"], how="inner")
        .merge(df_market_cap, on=["datetime", "instrument"], how="inner")
    )
    df_merged_predictions.rename(columns={"datetime": "Date", "instrument": "Ticker"}, inplace=True)

    # Merge only the Date, Ticker, and Return columns from df_all.
    df_temp = df_all[['Date', 'Ticker', 'Return']]

    # Merge on 'Date' and 'Ticker' columns (using left join to keep all rows in df_merged_predictions)
    df_merged_predictions = df_merged_predictions.drop(columns=['Actual_Return'], errors='ignore') \
        .merge(df_temp, on=['Date', 'Ticker'], how='left')

    # Rename the merged column to 'Actual_Return'
    df_merged_predictions.rename(columns={'Return': 'Actual_Return'}, inplace=True)


    # To CSV
    csv_path = "data/Output/predictions_with_sentiment.csv"
    df_merged_predictions.to_csv(csv_path, index=False)
    ######
       
######################################################################################

print("IC: {:.4f} pm {:.4f}".format(np.mean(ic), np.std(ic)))
print("ICIR: {:.4f} pm {:.4f}".format(np.mean(icir), np.std(icir)))
print("RIC: {:.4f} pm {:.4f}".format(np.mean(ric), np.std(ric)))
print("RICIR: {:.4f} pm {:.4f}".format(np.mean(ricir), np.std(ricir)))


###### 3
print(df_merged_predictions.head())

Model Loaded from model/model_training_with_sentiment_0.pkl
Epoch: Previously trained.
{'IC': np.float64(0.08734820246988247), 'ICIR': np.float64(0.813890699283661), 'RIC': np.float64(0.0749257299607596), 'RICIR': np.float64(0.664116963632146)}
IC: 0.0873 pm 0.0000
ICIR: 0.8139 pm 0.0000
RIC: 0.0749 pm 0.0000
RICIR: 0.6641 pm 0.0000
        Date Ticker  Predicted_Return       Price    Market_Cap  Actual_Return
0 2024-07-15   AAPL         -0.021814  234.399994  3.575710e+12       0.016743
1 2024-07-16   AAPL         -0.024711  234.820007  3.575710e+12       0.001792
2 2024-07-17   AAPL         -0.027905  228.880005  3.575710e+12      -0.025296
3 2024-07-18   AAPL         -0.026494  224.179993  3.575710e+12      -0.020535
4 2024-07-19   AAPL         -0.020865  224.309998  3.575710e+12       0.000580


In [118]:

# Read the CSV file
df = pd.read_csv("data/Output/TopN_LongShort_position.csv")

df['Date'] = pd.to_datetime(df['Date'])
df_merged_predictions['Date'] = pd.to_datetime(df_merged_predictions['Date'])




df = df.merge(
    df_merged_predictions[['Date', 'Ticker', 'Price']],
    on=['Date', 'Ticker'],
    how='left'
)



# Check the result
print(df.head())


        Date Ticker Position_Type      Shares  Weight       Price
0 2024-07-15   AVGO          Long  583.362508     0.2  171.419998
1 2024-07-15   AXON          Long  324.002058     0.2  308.640015
2 2024-07-15   PCAR          Long  956.480125     0.2  104.550003
3 2024-07-15    TXN          Long  495.638396     0.2  201.759995
4 2024-07-15   VRTX          Long  203.728224     0.2  490.850006


In [119]:
csv_path = "data/Output/Updated_Allocation.csv"
df.to_csv(csv_path, index=False)

In [25]:
import os
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL connection settings
db_user = "postgres"
db_password = "qf5214"
db_host = "134.122.167.14"
db_port = 5555
db_name = "QF5214"
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")



file_name = "data/Output/TopN_LongShort_Marketcap_position.csv"

# Check if the file exists in the current folder
if os.path.exists(file_name):
    print(f"File '{file_name}' found in the current directory.")
    # Load and display the first few rows of the CSV file
    df = pd.read_csv(file_name)
    print(df.head())
else:
    print(f"File '{file_name}' not found in the current directory: {os.getcwd()}")


# Define target table and schema
table_name = "dailytrading"
schema = "tradingstrategy"

# Insert DataFrame into PostgreSQL table
try:
    df.to_sql(
        name=table_name,
        con=engine,
        if_exists="append",  # Use "append" to add to the existing table, or "replace" to overwrite
        index=False,
        schema=schema
    )
    print(f"Data successfully stored in {schema}.{table_name}")
except Exception as e:
    print(f"Error storing data: {e}")


File 'data/Output/TopN_LongShort_Marketcap_position.csv' found in the current directory.
         Date Ticker Position_Type       Shares    Weight
0  2024-08-01   AMGN          Long   535.230302  0.359172
1  2024-08-01   CTSH          Long   582.242845  0.088862
2  2024-08-01   ODFL          Long   193.838933  0.078776
3  2024-08-01   ADBE          Long   377.669366  0.412725
4  2024-08-01    WBD          Long  3607.738467  0.060466
Error storing data: (psycopg2.errors.UndefinedColumn) column "Position_Type" of relation "dailytrading" does not exist
LINE 1: ...O tradingstrategy.dailytrading ("Date", "Ticker", "Position_...
                                                             ^

[SQL: INSERT INTO tradingstrategy.dailytrading ("Date", "Ticker", "Position_Type", "Shares", "Weight") VALUES (%(Date__0)s, %(Ticker__0)s, %(Position_Type__0)s, %(Shares__0)s, %(Weight__0)s), (%(Date__1)s, %(Ticker__1)s, %(Position_Type__1)s, %(Shares__1)s ... 91202 characters truncated ... __998)s), (%(

In [ ]:
from sqlalchemy import create_engine, text

# PostgreSQL connection settings
db_user = "postgres"
db_password = "qf5214"
db_host = "134.122.167.14"
db_port = 5555
db_name = "QF5214"
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

with engine.connect() as conn:
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS tradingstrategy"))
    conn.commit()

print("Schema 'tradingstrategy' has been created (if it did not already exist).")


Schema 'tradingstrategy' has been created (if it did not already exist).


In [27]:
import os
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL connection settings
db_user = "postgres"
db_password = "qf5214"
db_host = "134.122.167.14"
db_port = 5555
db_name = "QF5214"
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

file_name = "data/Output/TopN_LongShort_Marketcap_position.csv"

# Check if the file exists in the current folder
if os.path.exists(file_name):
    print(f"File '{file_name}' found in the current directory.")
    # Load and display the first few rows of the CSV file
    df = pd.read_csv(file_name)
    print(df.head())
else:
    print(f"File '{file_name}' not found in the current directory: {os.getcwd()}")

# Define target table and schema
table_name = "dailytrading"
schema = "tradingstrategy"

# Insert DataFrame into PostgreSQL table using "replace"
try:
    df.to_sql(
        name=table_name,
        con=engine,
        if_exists="replace",  # Overwrite the table if it already exists
        index=False,
        schema=schema
    )
    print(f"Data successfully stored in {schema}.{table_name} (table replaced if it existed).")
except Exception as e:
    print(f"Error storing data: {e}")


File 'data/Output/TopN_LongShort_Marketcap_position.csv' found in the current directory.
         Date Ticker Position_Type       Shares    Weight
0  2024-08-01   AMGN          Long   535.230302  0.359172
1  2024-08-01   CTSH          Long   582.242845  0.088862
2  2024-08-01   ODFL          Long   193.838933  0.078776
3  2024-08-01   ADBE          Long   377.669366  0.412725
4  2024-08-01    WBD          Long  3607.738467  0.060466
Data successfully stored in tradingstrategy.dailytrading (table replaced if it existed).


In [115]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL connection settings
db_user = "postgres"
db_password = "qf5214"
db_host = "134.122.167.14"
db_port = 5555
db_name = "QF5214"
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Define the SQL query to obtain the table from the specified schema and table name.
query = "SELECT * FROM tradingstrategy.dailytrading"

# Execute the query and load the result into a pandas DataFrame
df = pd.read_sql(query, engine)

# Display the first few rows of the DataFrame
print(df)


            Date Ticker Position_Type      Shares  Weight
0     2024-07-15   AVGO          Long  583.362508     0.2
1     2024-07-15   AXON          Long  324.002058     0.2
2     2024-07-15   PCAR          Long  956.480125     0.2
3     2024-07-15    TXN          Long  495.638396     0.2
4     2024-07-15   VRTX          Long  203.728224     0.2
...          ...    ...           ...         ...     ...
1785  2025-03-31   TSLA         Short         NaN     0.2
1786  2025-03-31   COST         Short         NaN     0.2
1787  2025-03-31   GILD         Short         NaN     0.2
1788  2025-03-31   BKNG         Short         NaN     0.2
1789  2025-03-31   AMGN         Short         NaN     0.2

[1790 rows x 5 columns]
